In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import torch.nn.functional as F
from sklearn.preprocessing import StandardScaler

train_df = pd.read_csv('./datasets/mitbih_dataset/mitbih_train.csv', header=None)
test_df = pd.read_csv('./datasets/mitbih_dataset/mitbih_test.csv', header=None)

X_train = train_df.iloc[:, :187].values
y_train = train_df.iloc[:, 187].values
X_test = test_df.iloc[:, :187].values
y_test = test_df.iloc[:, 187].values

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

batch_size = 1024

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
class Inception1D(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Inception1D, self).__init__()
        self.branch1 = nn.Conv1d(in_channels, out_channels, kernel_size=1)

        self.branch2 = nn.Sequential(
            nn.Conv1d(in_channels, out_channels, kernel_size=1),
            nn.Conv1d(out_channels, out_channels, kernel_size=3, padding=1)
        )

        self.branch3 = nn.Sequential(
            nn.Conv1d(in_channels, out_channels, kernel_size=1),
            nn.Conv1d(out_channels, out_channels, kernel_size=5, padding=2)
        )

        self.branch4 = nn.Sequential(
            nn.MaxPool1d(kernel_size=3, stride=1, padding=1),
            nn.Conv1d(in_channels, out_channels, kernel_size=1)
        )

    def forward(self, x):
        branch1 = self.branch1(x)
        branch2 = self.branch2(x)
        branch3 = self.branch3(x)
        branch4 = self.branch4(x)
        outputs = torch.cat([branch1, branch2, branch3, branch4], dim=1)
        return outputs

class InceptionNet1D(nn.Module):
    def __init__(self, num_classes=5):
        super(InceptionNet1D, self).__init__()
        self.initial_conv = nn.Conv1d(1, 32, kernel_size=3, stride=2, padding=1)

        self.inception1 = Inception1D(32, 32)
        self.inception2 = Inception1D(128, 64)
        self.inception3 = Inception1D(256, 128)
        self.inception4 = Inception1D(512, 256)

        self.pool = nn.AdaptiveAvgPool1d(1)
        self.fc = nn.Linear(1024, num_classes)

    def forward(self, x):
        x = F.relu(self.initial_conv(x))
        x = self.inception1(x)
        x = self.inception2(x)
        x = self.inception3(x)
        x = self.inception4(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

dummy_input = torch.randn(10, 1, 187)  # (batch_size, num_channels, seq_length)
model = InceptionNet1D()
output = model(dummy_input)
print(output.shape)  # (batch_size, num_classes)

train_df = pd.read_csv('./datasets/mitbih_dataset/mitbih_train.csv', header=None)
test_df = pd.read_csv('./datasets/mitbih_dataset/mitbih_test.csv', header=None)

X_train = train_df.iloc[:, :187].values
y_train = train_df.iloc[:, 187].values
X_test = test_df.iloc[:, :187].values
y_test = test_df.iloc[:, 187].values

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
batch_size = 1024

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

from tqdm import tqdm
lr = 0.001
num_epochs = 10

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = InceptionNet1D().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    with tqdm(total=len(train_loader), desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch') as pbar:
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.unsqueeze(1).to(device), labels.to(device)  # Ajout d'une dimension pour le canal

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            pbar.set_postfix(loss=running_loss/(i+1))
            pbar.update(1)

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

model.eval()
correct = 0
total = 0
with torch.no_grad():
    with tqdm(total=len(test_loader), desc='Evaluating', unit='batch') as pbar:
        for inputs, labels in test_loader:
            inputs, labels = inputs.unsqueeze(1).to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            pbar.update(1)

print(f"Accuracy: {100 * correct / total}%")


In [ ]:
import os
import sys
import torch
from torch.utils.data import TensorDataset, DataLoader
import subprocess
import numpy as np
import pandas as pd
import glob
from collections import OrderedDict
import random

# Clone and import CNNWordReco repository
if not os.path.isdir('CNNWordReco'):
    subprocess.call(['git', 'clone', 'https://github.com/saztorralba/CNNWordReco'])
if 'CNNWordReco' not in sys.path:
    sys.path.append('CNNWordReco')
from utils.cnn_func import load_data, train_model, validate_model, test_model
from test_wordreco import show_matrix

# Arguments
args = {
    'cv_percentage': 0.1,
    'xsize': 20,
    'ysize': 20,
    'num_blocks': 10,
    'channels': 32,
    'dropout': 0.3,
    'embedding_size': 128,
    'epochs': 20,
    'batch_size': 32,
    'learning_rate': 0.001,
    'seed': 0,
    'device': 'cpu',
    'verbose': 1,
    'augment': False,
    'vocab': OrderedDict({'ZERO': 0, 'ONE': 1, 'TWO': 2, 'THREE': 3, 'FOUR': 4, 'FIVE': 5, 'SIX': 6, 'SEVEN': 7, 'EIGHT': 8, 'NINE': 9})
}

# Initialise the random seeds
random.seed(args['seed'])
torch.manual_seed(args['seed'])
torch.cuda.manual_seed(args['seed'])
torch.backends.cudnn.deterministic = True

# Read data and store in dataframe
wavfiles = glob.glob('./datasets/recordings/*.wav')
speakers = [file.split('/')[-1].split('_')[1] for file in wavfiles]
words = [list(args['vocab'].keys())[int(file.split('/')[-1].split('_')[0])] for file in wavfiles]
rec_number = [int(file.split('/')[-1].split('_')[2].split('.')[0]) for file in wavfiles]
data = pd.DataFrame({'wavfile': wavfiles, 'speaker': speakers, 'word': words, 'rec_number': rec_number})

# Perform training as defined in https://github.com/Jakobovski/free-spoken-digit-dataset/
# Recordings [5-49] for training and recordings [0-4] for testing
print('Training model with recordings [5-49] from all speakers')
# Load data
train_data = data.loc[data['rec_number'] >= 5].reset_index(drop=True)
test_data = data.loc[data['rec_number'] < 5].reset_index(drop=True)
trainset, validset, trainlabels, validlabels = load_data(train_data, True, **args)
args['mean'] = torch.mean(trainset.float())
args['std'] = torch.std(trainset.float())

# Convert the pandas data to PyTorch tensors
X_train = torch.tensor(trainset, dtype=torch.float32)
y_train = torch.tensor(trainlabels, dtype=torch.long)

# Reshape the input tensor to the shape expected by conv1d: (batch_size, channels, width)
X_train = X_train.view(X_train.size(0), 1, -1)

# Create Tensor datasets
train_dataset = TensorDataset(X_train, y_train)

# Define batch size
batch_size = 512  # Choose the batch size you want to use

# Create DataLoaders
train_loader_voice = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Print the shape of a sample
sample_batch = next(iter(train_loader_voice))
print("Shape of input batch:", sample_batch[0].shape)
print("Shape of target batch:", sample_batch[1].shape)


In [ ]:
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Fonction pour extraire les caractéristiques
def extract_features(model, data_loader):
    features = []
    labels_list = []
    model.eval()
    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs = inputs.to(device)

            outputs = model(inputs)
            features.append(outputs)
            labels_list.append(labels)

    features = torch.cat(features)
    labels_list = torch.cat(labels_list)
    return features, labels_list

# Extract features and labels from test data
test_features, test_labels = extract_features(model, train_loader_voice)

# Calculate cosine similarity
distances = cosine_similarity(test_features.detach().cpu(), test_features.detach().cpu())
np.fill_diagonal(distances, float('-inf'))

# Fonction pour trouver les indices des éléments les plus similaires
def find_max_indices(arr, k):
    if k > len(arr):
        raise ValueError("k cannot be greater than the size of the array")
    idx = np.argpartition(arr, -k)[-k:]
    sorted_idx = np.argsort(arr[idx])[::-1]
    return idx[sorted_idx]

# Fonction pour calculer la précision top-k
def compute_top_k_accuracy(distances, test_labels, k):
    correct = 0
    for id_, elem in enumerate(distances):
        indices = find_max_indices(elem, k)
        candidates = [test_labels[idx].item() for idx in indices]
        if test_labels[id_].item() in candidates:
            correct += 1
    accuracy = correct / len(distances)
    return accuracy

# Compute top-k accuracy
real_top_accuracies = []
for k in range(1, 6):
    accuracy = compute_top_k_accuracy(distances, test_labels, k)
    real_top_accuracies.append(accuracy)
    print(f"Top-{k} : {accuracy:.4f}")
